In [ ]:
import pandas as pd
import numpy as np
from tabularepimdl.SimpleInfection import SimpleInfection
from tabularepimdl.EpiModel import EpiModel

In [2]:
class Runner():
    """Class to automate running multiple iterations of an EpiModel simulation."""
    
    def __init__(self, model: EpiModel, time_start: int | float = 0, time_end: int | float = 0, time_step: int | float = 0, 
                 reset_flag: bool = False, added_data: pd.DataFrame = None, added_time: list[int | float] = None) -> None:
        """
        Initialization.

        Parameters:
        @para model (EpiModel): The epidemiological model to run simulations on.
        @para time_start: the start time of model simulation.
        @para time_end: the end time of model simulation.
        @para time_step: the incremenal size of time stamp.
        @para reset_flag: Whether to reset the model before running.
        @para added_data: new data points to be added to the model's current state.
        @para added_time: the time point when the new data is added.
        """
        self.model = model
        self.time_start = time_start
        self.time_end = time_end
        self.time_step = time_step
        self.reset_flag = reset_flag
        self.added_data = added_data
        self.added_time = added_time

    def reset_model(self) -> None:
        """Resets the model if the reset_flag is True."""
        if self.reset_flag:
            self.model.reset()
        else:
            pass

    def add_new_data(self, added_time: int | float = 0) -> None:
        """Adds new data to the current state."""
        required_columns = {"T"} #added_data needs to have column T
        #print("going into add_new_data method.")
        
        if not required_columns.issubset(self.model.cur_state.columns): #raise error when adding data at t=0
            raise ValueError ("The initial epi data missed column T.")
        
        #if required_columns.issubset(self.model.cur_state.columns): print('there is T')
        #else: print('no T')
        if self.added_data is not None:
            if required_columns.issubset(self.added_data.columns) and added_time in self.added_data["T"].values: #check if the event time is present in added_data column T
                self.model.cur_state = pd.concat([ self.model.cur_state, self.added_data.loc[self.added_data["T"]==added_time] ], ignore_index=True)
                #print('add new data, cur_state is\n', self.model.cur_state)
            else: raise ValueError (f"The added data does not include record for time {added_time}. Please check the added data.")
        else:
            raise ValueError (f"The added data is None. Please check the added data.")

    def run(self) -> pd.DataFrame:
        """
        Runs the simulation over the specified time range.
        Returns: cur_state at the end of the simulation.
        """
        for t in np.arange(self.time_start, self.time_end, self.time_step):
            if self.added_time is not None and t in self.added_time:
                self.add_new_data(added_time=t) # Inject data if needed at this timestep
            
            self.model.do_timestep(dt=self.time_step)  # Run one timestep

        return self.model.cur_state #only return current state of the Epi model


In [6]:
population = pd.DataFrame({
    'InfState' : pd.Categorical(['S','I'], categories=['S','I','R']),
    'N' : [999, 1],
    'T': 0.0 #assign T with float number
})

In [4]:
t=[0.5, 1.0] #added_time
to_add = pd.DataFrame({
    'InfState' : pd.Categorical(['I', 'I'], categories=['S','I','R']),
    'N' : [1, 2],
    'T': [0.5, 1.0]
}) #added_data

In [7]:
infect_rule = SimpleInfection(beta=0.5, column='InfState')
determ_epi_mdl = EpiModel(init_state = population ,rules=[infect_rule])

In [8]:
r = Runner(determ_epi_mdl, time_start = 0, time_end=5, time_step=0.25, reset_flag=True, added_data=to_add, added_time=t)
r.reset_model()

In [9]:
r.run()

initial current_state of each dt is
   InfState    N    T
0        S  999  0.0
1        I    1  0.0
Epi model starts!!!
current ruleset is
 [SimpleInfection(stochastic=False, beta=0.5, column='InfState', s_st='S', i_st='I', inf_to='I', freq_dep=True)]
current rule is
 stochastic=False beta=0.5 column='InfState' s_st='S' i_st='I' inf_to='I' freq_dep=True
epi model rule based
nw_delta is
   InfState         N    T
0        S -0.124867  0.0
1        I  0.124867  0.0
all_deltas is
   InfState         N    T
0        S -0.124867  0.0
1        I  0.124867  0.0
finished current ruleset, moving on
before concat cur_state and all_deltas, cur_state is
   InfState    N    T
0        S  999  0.0
1        I    1  0.0
before concat cur_state and all_deltas, all_deltas is
   InfState         N    T
0        S -0.124867  0.0
1        I  0.124867  0.0
after concat but before grouping nw_state is
   InfState           N    T
0        S  999.000000  0.0
1        I    1.000000  0.0
0        S   -0.124867 

,InfState,N,T
0,I,31.230548,5.0
1,S,971.769452,5.0
